In [1]:
!pip install -q llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Helpful Imports / Logging

In [2]:
from llama_index.core.response.notebook_utils import display_response

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### LLM

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from google.colab import userdata
from llama_index.embeddings.openai import OpenAIEmbedding

open_ai_key = userdata.get('PHANH_API_KEY')
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1, api_key=open_ai_key)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", api_key=open_ai_key)

In [5]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('PHANH_API_KEY')

## Data processing

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Extract URLs from base url

In [ ]:
import requests
from bs4 import BeautifulSoup
# Set up headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to extract blog data
def extract_blog_data(base_url):
    try:
        r = requests.get(base_url, headers=headers, timeout=10)
        r.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(r.content, 'html.parser')

        blog_posts = soup.find_all('div', class_='CardBlog_card__mm0Zw')
        blogs = []

        for post in blog_posts:
            title_elem = post.find('p', class_='CardBlog_title__qC51U')
            date_elem = post.find('p', class_='Text_text__zPO0D Text_text-size-16__PkjFu')

            if title_elem and date_elem:
                title = title_elem.text.strip()
                date = date_elem.text.strip()
                url = title_elem.find('a')['href'] if title_elem.find('a') else 'No link'
                full_url = f"https://www.llamaindex.ai{url}"

                blogs.append({
                    'title': title,
                    'date': date,
                    'url': full_url
                })

        return blogs

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

base_url = 'https://www.llamaindex.ai/blog'
blog_data = extract_blog_data(base_url)

In [ ]:
blog_data

[{'title': 'LlamaIndex Newsletter 2024-07-16',
  'date': 'Jul 16, 2024',
  'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16'},
 {'title': 'Arize AI and LlamaIndex Roll Out Joint Platform for Evaluating LLM Applications',
  'date': 'Jul 11, 2024',
  'url': 'https://www.llamaindex.ai/blog/arize-ai-and-llamaindex-roll-out-joint-platform-for-evaluating-llm-applications'},
 {'title': 'Case study: Lyzr: Taking autonomous AI agents to $1M+ ARR with LlamaIndex',
  'date': 'Jul 10, 2024',
  'url': 'https://www.llamaindex.ai/blog/case-study-lyzr-taking-autonomous-ai-agents-to-usd1m-arr-with-llamaindex'},
 {'title': 'LlamaIndex Newsletter 2024-07-09',
  'date': 'Jul 9, 2024',
  'url': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-09'},
 {'title': 'LlamaCloud - Built for Enterprise LLM App Builders',
  'date': 'Jul 9, 2024',
  'url': 'https://www.llamaindex.ai/blog/llamacloud-built-for-enterprise-llm-app-builders'},
 {'title': 'LlamaIndex Newsletter 2024-07-0

#### Save data into file for reuse

In [ ]:
# Save blog data to a JSON file
import json
if blog_data:
    with open('/content/drive/MyDrive/VietAI/blog_data.json', 'w', encoding='utf-8') as f:
        json.dump(blog_data, f, ensure_ascii=False, indent=4)
    print(f"Blog data saved to blog_data.json")
else:
    print("No blog data to save.")

# Optionally, print the number of blog posts extracted
print(f"Number of blog posts extracted: {len(blog_data)}")

#### Load data from file

In [12]:
import json

def load_blog_data(file_path='blog_data.json'):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            blog_data = json.load(f)
        print(f"Successfully loaded {len(blog_data)} blog posts from {file_path}")
        return blog_data
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {file_path}. The file may be corrupted.")
        return []
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return []

# Load the blog data
blog_data = load_blog_data('/content/drive/MyDrive/VietAI/blog_data.json')

# If you want to see the data, you can print the first few entries
if blog_data:
    print("\nFirst few blog entries:")
    for entry in blog_data[:3]:  # Print first 3 entries
        print(f"Title: {entry['title']}")
        print(f"Date: {entry['date']}")
        print(f"URL: {entry['url']}")
        print()
else:
    print("No blog data was loaded.")

Successfully loaded 155 blog posts from /content/drive/MyDrive/VietAI/blog_data.json

First few blog entries:
Title: LlamaIndex Newsletter 2024-07-16
Date: Jul 16, 2024
URL: https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16

Title: Arize AI and LlamaIndex Roll Out Joint Platform for Evaluating LLM Applications
Date: Jul 11, 2024
URL: https://www.llamaindex.ai/blog/arize-ai-and-llamaindex-roll-out-joint-platform-for-evaluating-llm-applications

Title: Case study: Lyzr: Taking autonomous AI agents to $1M+ ARR with LlamaIndex
Date: Jul 10, 2024
URL: https://www.llamaindex.ai/blog/case-study-lyzr-taking-autonomous-ai-agents-to-usd1m-arr-with-llamaindex



# Basic RAG

## Create Documents from urls

#### Clean text

In [13]:
import re
def preprocess_text(text):
    if not text:  # Check if text is None or empty
        return "No content to process"
    # Normalize newlines
    text = re.sub(r'\s+', ' ', text)

    # Add space before capital letters that follow lowercase letters (for camelCase)
    #text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)

    # Remove any remaining HTML tags if any
    text = re.sub(r'<[^>]+>', '', text)

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #Lower
    #text = text.lower()

    return text.strip()


#### Extract documents from each URL

In [14]:
from llama_index.readers.web import BeautifulSoupWebReader
from llama_index.core import Document
from bs4 import BeautifulSoup
from llama_index.readers.web import SimpleWebPageReader

# Now, use BeautifulSoupWebReader to extract documents from each URL
reader = BeautifulSoupWebReader()
documents = []

for blog in blog_data:
    url = blog['url']

    try:
        # Load data from the URL
        loaded_document = reader.load_data([url])

        if loaded_document:
            doc = loaded_document[0] #return list of doc
            doc.text = preprocess_text(doc.text)
            # Update the metadata with additional information
            doc.metadata.update({
                'title': blog['title'],
                'date': blog['date'],
            })

            documents.append(doc)
        else:
            print(f"No content extracted from {url}")
    except Exception as e:
        print(f"Error extracting content from {url}: {e}")

print(f"Created {len(documents)} Document objects.")

# Optional: Print some information about the first document
if documents:
    print("\nFirst document:")
    print(f"Text preview: {documents[0].text[:500]}...")
    print(f"Metadata: {documents[0].metadata}")

Created 155 Document objects.

First document:
Text preview: LlamaIndex Newsletter 2024-07-16  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jul 16, 2024LlamaIndex Newsletter 2024-07-16Hello, Llama Family! Welcome to this weeks edition of the LlamaIndex newsletter! Were thrilled to share some exciting updates about our products, the implementation of GraphRAG, demos that have achieved over $1M in ARR, extensive guides, in-de...
Metadata: {'URL': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16', 'title': 'LlamaIndex Newsletter 2024-07-16', 'date': 'Jul 16, 2024'}


In [15]:
documents[0].text

'LlamaIndex Newsletter 2024-07-16  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jul 16, 2024LlamaIndex Newsletter 2024-07-16Hello, Llama Family! Welcome to this weeks edition of the LlamaIndex newsletter! Were thrilled to share some exciting updates about our products, the implementation of GraphRAG, demos that have achieved over $1M in ARR, extensive guides, in-depth tutorials, and hackathons.Before we get into the details of our newsletter, were thrilled to share the beta launch of LlamaCloud. This new data processing layer boosts RAG workflows with sophisticated parsing, indexing, and retrieval functions. Alongside this, were also introducing LlamaTrace in partnership with Arize AI, which provides unmatched tracing, observability, and evaluation capabilities for LLM application workflows.Signup here: cloud.llamaindex.ai The highlights:LlamaCloud Launc

## Chunking

In [41]:
# chunking
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)

Settings.text_splitter = splitter


In [42]:
nodes = splitter.get_nodes_from_documents(documents)
len(nodes)

412

In [43]:
nodes[2]

TextNode(id_='ee21d038-6fbe-49d9-b685-bb939e8eccbf', embedding=None, metadata={'URL': 'https://www.llamaindex.ai/blog/arize-ai-and-llamaindex-roll-out-joint-platform-for-evaluating-llm-applications', 'title': 'Arize AI and LlamaIndex Roll Out Joint Platform for Evaluating LLM Applications', 'date': 'Jul 11, 2024'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://www.llamaindex.ai/blog/arize-ai-and-llamaindex-roll-out-joint-platform-for-evaluating-llm-applications', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'URL': 'https://www.llamaindex.ai/blog/arize-ai-and-llamaindex-roll-out-joint-platform-for-evaluating-llm-applications', 'title': 'Arize AI and LlamaIndex Roll Out Joint Platform for Evaluating LLM Applications', 'date': 'Jul 11, 2024'}, hash='b4356f1e0fa67eb529549febac2ccace4f6a457b35caff1744912ff6008ecd12')}, text='Arize AI and LlamaIndex Roll Out Joint Platform for Evaluating LLM 

## Query with ChromaDb (Persitant index)

In [44]:
!pip install -q llama-index-vector-stores-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.3 MB/s eta 0:00:00

In [45]:
pwd

'/content'

In [46]:
cd /content/drive/MyDrive/VietAI

/content/drive/MyDrive/VietAI


In [47]:
from llama_index.embeddings.openai import OpenAIEmbedding
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import SummaryIndex

In [48]:
# Set up ChromaDB client (adjust the path to your Google Drive)
db_path = "./chroma_DB"
db = chromadb.PersistentClient(path=db_path)

db.delete_collection("llamaindex_collection_new")

# Get or create collection
chroma_collection = db.create_collection("llamaindex_collection_new")
# Set up vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Create a StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# Set up index
index = VectorStoreIndex(nodes, storage_context=storage_context)

In [49]:
# load
db_path = "./chroma_DB"
db2 = chromadb.PersistentClient(path=db_path)
chroma_collection = db2.get_or_create_collection("llamaindex_collection_new")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

chroma_index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)


## Get title and position of extracted text

In [79]:
def query_with_sources(query_engine=query_engine, query=""):

    # Query the index
    response = query_engine.query(query)
    print("Sources:")
    for node in response.source_nodes:

        # Get the title from metadata
        title = node.node.metadata.get('title', 'Unknown Title')

        # Get the URL from metadata
        url = node.node.metadata.get('URL', 'URL not available')

        # Get the full text of the node
        full_text = node.node.text


        print(f"Title: {title}")
        print(f" URL: {url}")
        print(f"Text preview: {full_text[:200]} \n\n")
    display_response(response)


## Answers

#### Refine

In [81]:
 # Define query engine
query_engine = index.as_query_engine(response_mode='refine')

In [82]:
response = query_with_sources(query_engine, "What are key features of llama-agents?")

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterp 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agent HYDE_PROMPT_STR = ( "Please rewrite the following query to include more detail:\n{query_str}\n" ) HYD 




**`Final Response:`** Distributed Service Oriented Architecture, Communication via standardized API interfaces, Define agentic and explicit orchestration flows, Ease of deployment, Scalability and resource management.

In [53]:
response = query_with_sources(query_engine,"What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook??")

Sources:
Title: OpenAI Cookbook: Evaluating RAG systems
 URL: https://www.llamaindex.ai/blog/openai-cookbook-evaluating-rag-systems-fe393c61fb93
Text preview: OpenAI Cookbook: Evaluating RAG systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 Ll 


Title: A Cheat Sheet and Some Recipes For Building Advanced RAG
 URL: https://www.llamaindex.ai/blog/a-cheat-sheet-and-some-recipes-for-building-advanced-rag-803a9d94c41b
Text preview: A Cheat Sheet and Some Recipes For Building Advanced RAG  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog T 




**`Final Response:`** The two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook are the Retrieval System and Response Generation.

In [54]:
response = query_with_sources(query_engine,"What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system")

Sources:
Title: Boosting RAG: Picking the Best Embedding & Reranker models
 URL: https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83
Text preview: Implemented by the user. """ return self._retrieve(query_bundle) async def aretrieve(self, str_or_query_bundle: QueryType) -&gt; List[NodeWithScore]: if isinstance(str_or_query_bundle, str): str_or_qu 


Title: Boosting RAG: Picking the Best Embedding & Reranker models
 URL: https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83
Text preview: Nearly all embeddings benefit from reranking, showing improved hit rates and MRRs.Rerankers, especially CohereRerank, have demonstrated their capability to transform any embedding into a competitive o 




**`Final Response:`** The two main metrics used to evaluate the performance of the different rerankers in the RAG system are Mean Reciprocal Rank (MRR) and Hit Rate.

### Tree Summarize

In [58]:
 # Define query engine
query_engine = chroma_index.as_query_engine(response_mode='tree_summarize')

In [59]:
esponse = query_with_sources(query_engine,"What are key features of llama-agents?")

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterp 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agent HYDE_PROMPT_STR = ( "Please rewrite the following query to include more detail:\n{query_str}\n" ) HYD 




**`Final Response:`** The key features of llama-agents include a Distributed Service Oriented Architecture where each agent can function as an independently running microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment with the capability to launch, scale, and monitor each agent and control plane independently, and scalability and resource management through built-in observability tools to monitor system quality and performance.

In [ ]:
response = query_with_sources(query_engine,"What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook??")

Sources:
Title: OpenAI Cookbook: Evaluating RAG systems
 URL: https://www.llamaindex.ai/blog/openai-cookbook-evaluating-rag-systems-fe393c61fb93
Text preview: openai cookbook: evaluating rag systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterpriseopen sourcecommunitycareersblog talk to us 2024 ll 


Title: A Cheat Sheet and Some Recipes For Building Advanced RAG
 URL: https://www.llamaindex.ai/blog/a-cheat-sheet-and-some-recipes-for-building-advanced-rag-803a9d94c41b
Text preview: irrelevant information).llamaindex information compression recipe (notebook guide):from llama_index import simpledirectoryreader, vectorstoreindex from llama_index.query_engine import retrieverqueryen 




**`Final Response:`** The two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook are the retrieval system and response generation.

In [ ]:
response = query_with_sources(query_engine,"What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system")

Sources:
Title: Boosting RAG: Picking the Best Embedding & Reranker models
 URL: https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83
Text preview: however, actual outcomes may differ based on data characteristics, dataset size, and other variables like chunk_size, similarity_top_k, and so on.the table below showcases the evaluation results based 


Title: Boosting RAG: Picking the Best Embedding & Reranker models
 URL: https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83
Text preview: boosting rag: picking the best embedding & reranker models  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterpriseopen sourcecommunitycareersblog 




**`Final Response:`** The two main metrics used to evaluate the performance of the different rerankers in the RAG system are hit rate and mean reciprocal rank (MRR).

## Multiple index with persistant docstore

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#putting nodes in the docstore
# this allows you to define multiple indices over the same underlying docstore, instead of duplicating data across indices.
from llama_index.core.storage.docstore import SimpleDocumentStore

doc_store = SimpleDocumentStore()
doc_store.add_documents(nodes)


In [ ]:
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex
from llama_index.core import SummaryIndex
from llama_index.core import ComposableGraph

storage_context = StorageContext.from_defaults(docstore=doc_store)
#creating three different indices that all reference the same docstore through the storage context.
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [ ]:
# Set index id
summary_index.set_index_id("summary_index")
vector_index.set_index_id("vector_index")
keyword_table_index.set_index_id("keyword_table_index")

In [ ]:
# Persist the overall storage_context
storage_context.persist(persist_dir="/content/drive/MyDrive/VietAI/index_store/overall_context")

In [ ]:
# NOTE: the docstore sitll has the same nodes
len(storage_context.docstore.docs)

226

In [ ]:
# try to load index from storage
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [ ]:
# try vector_index
index = load_index_from_storage(storage_context, index_id="vector_index")

query_engine = index.as_query_engine()
response = query_with_sources("What are key features of llama-agents?")

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp 




**`Final Response:`** The key features of llama-agents include a distributed service-oriented architecture where each agent can function as an independent microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment allowing for launching, scaling, and monitoring each agent and the control plane independently, as well as built-in observability tools for monitoring the quality and performance of the system and individual agent services.

### Load multiple indices from storage

In [91]:
from llama_index.core import StorageContext
# Path where you saved your index
persist_dir = "/content/drive/MyDrive/VietAI/index_store/overall_context"

# Load the storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

from llama_index.core import get_response_synthesizer
# try to load load indexes from storage
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)


In [92]:
index = load_index_from_storage(storage_context, index_id="keyword_table_index")
query_engine = index.as_query_engine()
response = query_with_sources(query_engine, "What are key features of llama-agents?")

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp 


Title: LlamaIndex Newsletter 2024-07-02
 URL: https://www.llamaindex.ai/blog/llamaindex-newsl

**`Final Response:`** The key features of llama-agents include:
1. Distributed service-oriented architecture.
2. Communication via standardized API interfaces.
3. Defining agentic and explicit orchestration flows.
4. Ease of deployment for launching, scaling, and monitoring each agent and the control plane independently.
5. Scalability and resource management with built-in observability tools to monitor system performance.

# Test

In [ ]:
# enter your question here
question1 = ""

In [ ]:
question2 = ""

In [ ]:
index = load_index_from_storage(storage_context, index_id="vector_index")
query_engine = index.as_query_engine()
response = query_with_sources(query_engine, question1)

In [ ]:
response = query_with_sources(query_engine, question2)

# Avanced Rag

## Different chunking techniques

#### TokenTextSplitter

In [39]:
from llama_index.core.node_parser import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)
len(nodes)

401

In [40]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("What are key features of llama-agents?")

display_response(response)

**`Final Response:`** The key features of llama-agents include a Distributed Service Oriented Architecture where each agent can function as an independently running microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment with the capability to launch, scale, and monitor each agent and control plane independently, as well as scalability and resource management through built-in observability tools to monitor system performance and quality.

#### HierarchicalNodeParser
 It chunks input data into several layers of hierarchy, with each node linked to its parent. This structure is particularly useful when combined with the AutoMergingRetriever, as it facilitates the automatic replacement of retrieved nodes with their parent nodes when a majority of child nodes are retrieved. This process ensures that the LLM has access to a more complete context for response synthesis, enhancing the accuracy and relevance of the output.

In [60]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[3048, 2048, 1000]
)

nodes = node_parser.get_nodes_from_documents(documents)
len(nodes)

914

In [61]:
nodes[0]

TextNode(id_='34dea513-3d1b-45d3-a1b1-8f2706522e33', embedding=None, metadata={'URL': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16', 'title': 'LlamaIndex Newsletter 2024-07-16', 'date': 'Jul 16, 2024'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'URL': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16', 'title': 'LlamaIndex Newsletter 2024-07-16', 'date': 'Jul 16, 2024'}, hash='9cd4f2088c78da703ada5910cfb265b0d37b6f8a35ff7318a16eb083db666369'), <NodeRelationship.CHILD: '5'>: [RelatedNodeInfo(node_id='9bf86a64-b01a-4dee-bb09-33e157efdcb4', node_type=<ObjectType.TEXT: '1'>, metadata={'URL': 'https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-16', 'title': 'LlamaIndex Newsletter 2024-07-16', 'date': 'Jul 16, 2024'}, hash='9cd4f2088c78da703ad

AutoMergingRetriever, which looks at a set of leaf nodes and recursively "merges" subsets of leaf nodes that reference a parent node beyond a given threshold. This allows us to consolidate potentially disparate, smaller contexts into a larger context that might help synthesis.

In [62]:
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

482

In [64]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

184

We then load these nodes into storage. The leaf nodes are indexed and retrieved via a vector store - these are the nodes that will first be directly retrieved via similarity search. The other nodes will be retrieved from a docstore.

In [67]:
## Load index into vector index
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

# We then define a VectorStoreIndex containing just the leaf-level nodes.
base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
)

In [68]:
base_retriever = base_index.as_retriever(similarity_top_k=6) # Without Automerging
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

query_str = ("What are key features of llama-agents?")

nodes = retriever.retrieve(query_str)
base_nodes = base_retriever.retrieve(query_str) # Without Automerging
print(len(nodes)) #combines closely related or overlapping information into single, more comprehensive nodes. So may be fewer nodes
print(len(base_nodes))

> Merging 2 nodes into parent node.
> Parent node id: 4d7dabcf-ee2e-41af-90a1-3f428d597844.
> Parent node text: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  Ll...

> Merging 1 nodes into parent node.
> Parent node id: e81e093e-bf96-419a-9acd-aea6a0265a5d.
> Parent node text: His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agen...

> Merging 1 nodes into parent node.
> Parent node id: 80ab5c6d-b8f9-46fa-aac6-7f1deec68513.
> Parent node text: LlamaIndex Newsletter 2024-07-02  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen s...

> Merging 2 nodes into parent node.
> Parent node id: df506443-a581-4826-9d00-63229ca12281.
> Parent node text: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  Ll...

> Merging 1 nodes into parent node.
> Parent node id: 71a33e83-9fdc-4ac6-a397-c046a2e638d4.
> Parent node text: LlamaIndex Newsletter 2024-07-02

In [70]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

**Node ID:** df506443-a581-4826-9d00-63229ca12281<br>**Similarity:** 0.6661840085211621<br>**Text:** Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jun 26, 2024Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI SystemsAgentsWe're excited to announce the alpha release of llama-agents, a new open-source framework designed to simplify the process of building, iterating, and deploying multi-agent AI systems and turn your agents into production microservices. Whether you're working on complex question-answering systems, collaborative AI assistants, or distributed AI workflows, llama-agents provides the tools and structure you need to bring your ideas to life.Key Features of llama-agentsDistributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue.Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an agentic orchestrator that decides which agents are relevant to the task.Ease of deployment: launch, scale and monitor each agent and your control plane independently.Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent serviceLet's dive into how you can start using llama-agents to build your own multi-agent systems.Getting Started with llama-agentsFirst, install the framework using pip:pip install llama-agents llama-index-agent-openaiBasic System SetupHere's a simple example of how to set up a basic multi-agent system using llama-agents. First well bring in our dependencies and set up our control plane, which contains our LLM-powered orchestratorimport dotenv dotenv.load_dotenv() # our .env file defines OPENAI_API_KEY from llama_agents import ( AgentService, ControlPlaneServer, SimpleMessageQueue, AgentOrchestrator, ) from llama_index.core.agent import FunctionCallingAgentWorker from llama_index.core.tools import FunctionTool from llama_index.llms.openai import OpenAI import logging # turn on logging so we can see the system working logging.getLogger("llama_agents").setLevel(logging.INFO) # Set up the message queue and control plane message_queue = SimpleMessageQueue() control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=AgentOrchestrator(llm=OpenAI()), )Next we create our tools using LlamaIndexs existing abstractions, provide those tools to an agent, and turn that agent into an independent microservice:# create a tool def get_the_secret_fact() -> str: """Returns the secret fact.""" return "The secret fact is: A baby llama is called a 'Cria'." tool = FunctionTool.from_defaults(fn=get_the_secret_fact) # Define an agent worker = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI()) agent = worker.as_agent() # Create an agent service agent_service = AgentService( agent=agent, message_queue=message_queue, description="General purpose assistant", service_name="assistant", )Finally we launch the service and the control plane. Note that here were using a helper function to run a single query through the system and then exit; next well show how to deploy this to production.# Set up the launcher for local testing from llama_agents import LocalLauncher launcher = LocalLauncher( [agent_service], control_plane, message_queue, ) # Run a single query through the system result = launcher.launch_single("What's the secret fact?") print(result)Deploying Your Multi-Agent SystemOnce you've tested your system locally, you can deploy it as a set of services for real production use. Here's how you might set that up. This is similar to the previous example, but weve added a second agent service and were using a different launcher. Lets bring in our dependencies and set up our control plane again:import dotenv dotenv.load_dotenv() from llama_agents import ( AgentService, AgentOrchestrator, ControlPlaneServer, SimpleMessageQueue, ) from llama_index.core.agent import FunctionCallingAgentWorker from llama_index.core.tools import FunctionTool from llama_index.llms.openai import OpenAI import logging # change logging level to enable or disable more verbose logging logging.getLogger("llama_agents").setLevel(logging.INFO) # create our multi-agent framework components message_queue = SimpleMessageQueue() control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=AgentOrchestrator(llm=OpenAI()), )Then as before we create a tool and an agent, though this time well add a second agent:# create a tool def get_the_secret_fact() -> str: """Returns the secret fact.""" return "The secret fact is: A baby llama is called a 'Cria'." tool = FunctionTool.from_defaults(fn=get_the_secret_fact) # create our agents worker1 = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI()) worker2 = FunctionCallingAgentWorker.from_tools([], llm=OpenAI()) agent1 = worker1.as_agent() agent2 = worker2.as_agent()We turn those agents into services:agent_server_1 = AgentService( agent=agent1, message_queue=message_queue, description="Useful for getting the secret fact.", service_name="secret_fact_agent", host="localhost", port=8003 ) agent_server_2 = AgentService( agent=agent2, message_queue=message_queue, description="Useful for getting random dumb facts.", service_name="dumb_fact_agent", host="localhost", port=8004 )And finally we launch each service as an independent agent. Here were doing them all from a single script, but each of these could be a totally separate service, launched and scaled independently:from llama_agents import ServerLauncher, CallableMessageConsumer # Additional human consumer def handle_result(message) -> None: print(f"Got result:", message.data) # the final result is published to a "human" consumer # so we define one to handle it! human_consumer = CallableMessageConsumer( handler=handle_result, message_type="human" ) # Define Launcher launcher = ServerLauncher( [agent_server_1, agent_server_2], control_plane, message_queue, additional_consumers=[human_consumer] ) launcher.launch_servers()Real-time monitoringOne of the coolest debugging features of our multi-agent system is our agent monitor, which is built right in. You launch it like this:llama-agents monitor --control-plane-url http://127.0.0.1:8000Once launched, you get an intuitive, point-and-click terminal application. You can see both of the agents running, and at the bottom you can inject a task like the query What is the secret fact? Youll get a job ID which you can then click on to see your results:Building a Query Rewriting RAG SystemLet's look at a more complex example: a Query Rewriting RAG system. This system will rewrite user queries to improve retrieval, then use the rewritten query to perform RAG over a document.This example demonstrates how to create a more sophisticated system that combines query rewriting with RAG to improve question-answering capabilities. See this notebook for a fuller explanation of whats going on.import dotenv dotenv.load_dotenv() # our .env defines OPENAI_API_KEY from llama_index.core import VectorStoreIndex, Document from llama_index.core.agent import FnAgentWorker from llama_index.core import PromptTemplate from llama_index.core.query_pipeline import QueryPipeline from llama_index.core.query_engine import RetrieverQueryEngine from llama_agents import ( AgentService, ControlPlaneServer, SimpleMessageQueue, PipelineOrchestrator, ServiceComponent, ) from llama_agents.launchers import LocalLauncher from llama_index.llms.openai import OpenAI import logging # change logging level to enable or disable more verbose logging logging.getLogger("llama_agents").setLevel(logging.INFO) # Load and index your document docs = [Document(text="The rabbit is a small mammal with long ears and a fluffy tail. His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agent HYDE_PROMPT_STR = ( "Please rewrite the following query to include more detail:\n{query_str}\n" ) HYDE_PROMPT_TMPL = PromptTemplate(HYDE_PROMPT_STR) def run_hyde_fn(state): prompt_tmpl, llm, input_str = ( state["prompt_tmpl"], state["llm"], state["__task__"].input, ) qp = QueryPipeline(chain=[prompt_tmpl, llm]) output = qp.run(query_str=input_str) state["__output__"] = str(output) return state, True hyde_agent = FnAgentWorker( fn=run_hyde_fn, initial_state={"prompt_tmpl": HYDE_PROMPT_TMPL, "llm": OpenAI()} ).as_agent() # Define a RAG agent def run_rag_fn(state): retriever, llm, input_str = ( state["retriever"], state["llm"], state["__task__"].input, ) query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm) response = query_engine.query(input_str) state["__output__"] = str(response) return state, True rag_agent = FnAgentWorker( fn=run_rag_fn, initial_state={"retriever": index.as_retriever(), "llm": OpenAI()} ).as_agent() # Set up the multi-agent system message_queue = SimpleMessageQueue() query_rewrite_service = AgentService( agent=hyde_agent, message_queue=message_queue, description="Query rewriting service", service_name="query_rewrite", ) rag_service = AgentService( agent=rag_agent, message_queue=message_queue, description="RAG service", service_name="rag", ) # Create the pipeline pipeline = QueryPipeline(chain=[ ServiceComponent.from_service_definition(query_rewrite_service), ServiceComponent.from_service_definition(rag_service), ]) orchestrator = PipelineOrchestrator(pipeline) control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=orchestrator, ) # Set up the launcher launcher = LocalLauncher( [query_rewrite_servi...<br>

**Node ID:** 3058ab62-4cec-4b57-a570-cbc63c07b0be<br>**Similarity:** 0.63911602171461<br>**Text:** For instance, you may want to define a specific workflow over text-to-SQL with our NLStructStoreQueryEngine (constrained), or a router module to decide between semantic search or summarization (less constrained), or use our SubQuestionQueryEngine to decompose a question among sub-documents (even less constrained).By default, agent loops are unconstrained, and can theoretically reason over any set of tools that you give them. This means that you can get out-of-the-box advanced search/retrieval capabilities  for instance, in our OpenAI cookbook we show that you can get joint text-to-SQL capabilities by simply providing a SQL query engine and Vector Store Query engine as tools. But on the other hand, agents built in this fashion can be quite unreliable (see our blog post for more insights). If you are using agents for search/retrieval, be mindful of the 1) LLM you pick, and the 2) set of tools you pick too.How are LlamaIndex data agents different than existing agent frameworks (LangChain, Hugging Face, etc.)?Most of these core concepts are not new. Our overall design has taken inspiration from popular tools and frameworks for building agents. But in our data agents design, weve tried our best to answer the following key questions well:How do we effectively index/query and retrieve data beforehand?How do we effectively index/query and retrieve data on the fly?How do we design API interfaces for read/writes that are simultaneously rich (can take in structured inputs), but also easy for agents to understand?How do we properly get sources in citations?Our goal with data agents is to create automated knowledge workers that can reason over and interact with data. Our core toolkit provides the foundations for properly indexing, retrieving, and querying data  these can be easily integrated as tools. We provide some additional tool abstractions to handle the cases where you want to cache API outputs on the fly (see above). Finally, we provide principled tool abstractions and design principles so that agents can interface with external services in a structured manner.Can I use Tools with LangChain agents? You can easily use any of our tools with LangChain agents as well.tools = tool_spec.to_tool_list() langchain_tools = [t.to_langchain_tool() for t in tools]See our tools usage guide for more details!ConclusionIn summary, today we launched two key items: Data Agent components (incl. agent reasoning loop and tool abstractions) and the LlamaHub Tool repository.ResourcesWeve written a comprehensive section in the docs  take a look here: https://gpt-index.readthedocs.io/en/latest/core_modules/agent_modules/agents/root.htmlTake a look at our LlamaHub Tools section: https://llamahub.ai/Notebook Tutorials for LlamaHub Tools: https://github.com/emptycrown/llama-hub/tree/main/llama_hub/tools/notebooksIf you have questions, please hop on our Discord: https://discord.gg/dGcwcsnxhURelated articlesBuilding a multi-agent concierge system2024-07-17Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems2024-06-26Automate online tasks with MultiOn and LlamaIndex2024-05-23Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations2024-03-19LlamaIndexBlogPartnersCareersContactStatusEnterpriseLlamaCloudLlamaParseOpen SourcePython packagePython docsTypeScript packageTypeScript docsLlamaHubGitHubCommunityNewsletterDiscordTwitter/XLinkedInYouTubeStarter projectscreate-llamaSEC InsightsChat LlamaIndexLlamaBotRAG CLI 2024 LlamaIndexPrivacy NoticeTerms of Service<br>

**Node ID:** 3b9739f3-af80-476a-a621-a69d4e1a5540<br>**Similarity:** 0.6332048798838363<br>**Text:** Data Agents  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexJerry Liu  Jul 12, 2023Data AgentsLlamaindexNLPAIAgentsDataToday were incredibly excited to announce the launch of a big new capability within LlamaIndex: Data Agents.Data Agents are LLM-powered knowledge workers that can intelligently perform various tasks over your data, in both a read and write function. They are capable of the following:Perform automated search and retrieval over different types of data  unstructured, semi-structured, and structured.Calling any external service API in a structured fashion. They can either process the response immediately, or index/cache this data for future use.Storing conversation history.Using all of the above to fulfill both simple and complex data tasks.Weve worked hard to provide abstractions, services, and guides on both the agents side and tools side in order to build data agents. Todays launch consists of the following key components:General Agent/Tool Abstractions: a set of abstractions to build agent loops, and to have those loops interact with tools according to a structured API definition.LlamaHub Tool Repository: A brand-new section within LlamaHub that consists of 15+ Tools (e.g. Google Calendar, Notion, SQL, OpenAPI) that can be connected. Opening to community contributions!See below for full details. We show you how to build a Gmail agent thats able to automatically create/send emails in <10 lines of code!ContextOur core mission at LlamaIndex is to unlock the full capabilities of LLMs over your external sources of data. It provides a set of tools to both define state (how to parse/structure your data), and compute (how to query your data). Up until now, our framework has primarily focused on search and retrieval use case. We have an incredible suite of tools and capabilities that not only allow you to create the basic RAG stack around a vector database + top-k retrieval, but also offer much greater functionality beyond that.A lot of that technology used to lie in our query engines. Our goal was to increase the capability of query engines to answer a wide range of different queries. In order to do this, we had to improve the reasoning capabilities of these query engines. As a result some of our existing query capabilities contain agent-like components: we have query engines capable of chain-of-thought reasoning, query decomposition, and routing. In the process, users had the option of choosing from a spectrum of query engines that had more constrained reasoning capabilities to less constrained capabilities.But there was a huge opportunity for LLMs to have an even richer set of interactions with data; they should be capable of general reasoning over any set of tools, whether from a database or an API. They should also be capable of both read and write capabilities  the ability to not only understand state but also modify it. As a result they should be able to do more than search and retrieval from a static knowledge source.Some existing services, toolkits, and research papers have already demonstrated the possibilities of LLM-powered agents that can interact with the external environment. Using these existing approaches as inspiration, we saw an opportunity to build a principled series of abstractions enabling anyone to build knowledge workers over their data.Core Components of Data AgentsBuilding a data agent requires the following core components:A reasoning loopTool abstractionsAt a high-level, a data agent is provided with a set of APIs, or Tools, to interact with. These APIs can return information about the world, or perform an action that modifies state. Each Tool exposes a request/response interface. The request is a set of structured parameters, and the response can be any format (at least conceptually, in most cases the response here is a text string of some form).Given an input task, the data agent uses a reasoning loop to decide which tools to use, in which sequence, and the parameters to call each tool.<br>

**Node ID:** 71a33e83-9fdc-4ac6-a397-c046a2e638d4<br>**Similarity:** 0.6283408263769629<br>**Text:** LlamaIndex Newsletter 2024-07-02  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jul 2, 2024LlamaIndex Newsletter 2024-07-02Hello, Llama enthusiasts! Welcome to this weeks edition of the LlamaIndex newsletter! In this issue, were excited to bring you exciting updates about llama-agents, live demos, extensive guides, and in-depth tutorials to enhance your understanding of our tools.Before moving into our newsletter, we have an exciting update on our enterprise offerings. We are thrilled to announce the waitlist release of LlamaCloud, our fully-managed ingestion service. Sign up now if youre eager to collaborate and build LLM applications with LlamaCloud. The highlights:Launched Llama-Agents Framework: Our new alpha-release, llama-agents, enables multi-agent AI systems for production with a distributed architecture, HTTP API communication, and agentic orchestration. Its designed for easy deployment, scalability, and observability. Blogpost, Tweet.create-llama Integrated with LlamaCloud: Streamline your LLM application data pipelines with create-llama, now integrated with LlamaCloud for faster setup and efficient system maintenance. Tweet. Feature Releases and Enhancements:We have launched llama-agents - new alpha-release framework that enables multi-agent AI systems to go into production. It features a distributed, service-oriented architecture, communication through standard HTTP APIs, agentic orchestration of flows, and is designed for easy deployment, scalability, and observability. Blogpost, Tweet.create-llama is now integrated with LlamaCloud to streamline the setup and management of data pipelines for LLM applications, providing a fast and efficient way to deploy and maintain these systems. Tweet.We have integrated with DSPy for Optimized RAG by utilizing DSPys optimization capabilities with LlamaIndexs data tools to enhance your query pipelines, optimize prompts, or repurpose DSPy predictors. Cookbook, Tweet. Demos:Automating Code Reviews, project by Composio with LlamaIndex automates code reviews using an AI agent in under 100 lines of code that monitors GitHub PRs, reviews them immediately upon creation, and posts feedback directly to your Slack channel. Codebase. Guides:Guide to Building an Agentic RAG Service with our comprehensive notebook that walks you through creating vector indexes, transforming them into query engines, turning each engine into a tool, providing these tools to agents, and launching the agents as services.Guide to AI Agents with LlamaIndex: Andreis comprehensive workshop from Gen AI Philippines, showcasing LLM applications through LlamaIndex. This beginner-friendly session covers topics from RAG to multi-hop agents. Video, Notebook. Tutorials:Kingzzms tutorial on crafting a custom hybrid retriever using LlamaIndexs flexible abstractions. This tutorial teaches you how to integrate full text and dense search capabilities from Elastic, and how to write your own reciprocal rank fusion function for optimal retrieval strategy.Jeffs tutorial on which outlines the essential tools needed to construct a report generator using a ReAct agent. Learn how to integrate a RAG tool over guideline documents, a web search tool, and a report generation tool that converts markdown text into PDFs.1littlecoders tutorial on llama-agents provides a detailed introduction to transforming multi-agent systems into microservices for production, including setup examples and a walkthrough of the architecture involving the control plane, message queue, and agent services using LlamaIndex abstractions.Mervin Praisons tutorial on the llama-agents framework provides a concise guide to setting up agent services, from notebook synchronization to server-client interactions, complete with over 10 practical examples.LlamaIndexBlogPartnersCareersContactStatusEnterpriseLlamaCloudLlamaParseOpen SourcePython packagePython docsTypeScript packageTypeScript docsLlamaHubGitHubCommunityNewsletterDiscordTwitter/XLinkedInYouTubeStarter projectscreate-llamaSEC InsightsChat LlamaIndexLlamaBotRAG CLI 2024 LlamaIndexPrivacy NoticeTerms of Service<br>

In [71]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** ec696876-3d24-443a-8d8b-82043e01c9ce<br>**Similarity:** 0.6919124729110046<br>**Text:** Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jun 26, 2024Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI SystemsAgentsWe're excited to announce the alpha release of llama-agents, a new open-source framework designed to simplify the process of building, iterating, and deploying multi-agent AI systems and turn your agents into production microservices. Whether you're working on complex question-answering systems, collaborative AI assistants, or distributed AI workflows, llama-agents provides the tools and structure you need to bring your ideas to life.Key Features of llama-agentsDistributed Service Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue.Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an agentic orchestrator that decides which agents are relevant to the task.Ease of deployment: launch, scale and monitor each agent and your control plane independently.Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent serviceLet's dive into how you can start using llama-agents to build your own multi-agent systems.Getting Started with llama-agentsFirst, install the framework using pip:pip install llama-agents llama-index-agent-openaiBasic System SetupHere's a simple example of how to set up a basic multi-agent system using llama-agents. First well bring in our dependencies and set up our control plane, which contains our LLM-powered orchestratorimport dotenv dotenv.load_dotenv() # our .env file defines OPENAI_API_KEY from llama_agents import ( AgentService, ControlPlaneServer, SimpleMessageQueue, AgentOrchestrator, ) from llama_index.core.agent import FunctionCallingAgentWorker from llama_index.core.tools import FunctionTool from llama_index.llms.openai import OpenAI import logging # turn on logging so we can see the system working logging.getLogger("llama_agents").setLevel(logging.INFO) # Set up the message queue and control plane message_queue = SimpleMessageQueue() control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=AgentOrchestrator(llm=OpenAI()), )Next we create our tools using LlamaIndexs existing abstractions, provide those tools to an agent, and turn that agent into an independent microservice:# create a tool def get_the_secret_fact() -> str: """Returns the secret fact.""" return "The secret fact is: A baby llama is called a 'Cria'." tool = FunctionTool.from_defaults(fn=get_the_secret_fact) # Define an agent worker = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI()) agent = worker.as_agent() # Create an agent service agent_service = AgentService( agent=agent, message_queue=message_queue, description="General purpose assistant", service_name="assistant", )Finally we launch the service and the control plane. Note that here were using a helper function to run a single query through the system and then exit; next well show how to deploy this to production.# Set up the launcher for local testing from llama_agents import LocalLauncher launcher = LocalLauncher( [agent_service], control_plane, message_queue, ) # Run a single query through the system result = launcher.launch_single("What's the secret fact?") print(result)Deploying Your Multi-Agent SystemOnce you've tested your system locally, you can deploy it as a set of services for real production use. Here's how you might set that up. This is similar to the previous example, but weve added a second agent service and were using a different launcher.<br>

**Node ID:** 6ba7db28-55e5-46b8-aec2-3fa5ea29d495<br>**Similarity:** 0.6707382559525482<br>**Text:** His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agent HYDE_PROMPT_STR = ( "Please rewrite the following query to include more detail:\n{query_str}\n" ) HYDE_PROMPT_TMPL = PromptTemplate(HYDE_PROMPT_STR) def run_hyde_fn(state): prompt_tmpl, llm, input_str = ( state["prompt_tmpl"], state["llm"], state["__task__"].input, ) qp = QueryPipeline(chain=[prompt_tmpl, llm]) output = qp.run(query_str=input_str) state["__output__"] = str(output) return state, True hyde_agent = FnAgentWorker( fn=run_hyde_fn, initial_state={"prompt_tmpl": HYDE_PROMPT_TMPL, "llm": OpenAI()} ).as_agent() # Define a RAG agent def run_rag_fn(state): retriever, llm, input_str = ( state["retriever"], state["llm"], state["__task__"].input, ) query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm) response = query_engine.query(input_str) state["__output__"] = str(response) return state, True rag_agent = FnAgentWorker( fn=run_rag_fn, initial_state={"retriever": index.as_retriever(), "llm": OpenAI()} ).as_agent() # Set up the multi-agent system message_queue = SimpleMessageQueue() query_rewrite_service = AgentService( agent=hyde_agent, message_queue=message_queue, description="Query rewriting service", service_name="query_rewrite", ) rag_service = AgentService( agent=rag_agent, message_queue=message_queue, description="RAG service", service_name="rag", ) # Create the pipeline pipeline = QueryPipeline(chain=[ ServiceComponent.from_service_definition(query_rewrite_service), ServiceComponent.from_service_definition(rag_service), ]) orchestrator = PipelineOrchestrator(pipeline) control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=orchestrator, ) # Set up the launcher launcher = LocalLauncher( [query_rewrite_service, rag_service], control_plane, message_queue, ) # Run a query result = launcher.launch_single("Tell me about rabbits") print(result)Public roadmapThis is an alpha release, meaning that wed love your feedback on features to better help you build multi-agent systems in production! Weve created a public roadmap showing where we plan to go from here. Were actively seeking public feedback on what works for you and what doesnt.Dive in!llama-agents provides a powerful, flexible framework for building complex multi-agent AI systems. Whether you're prototyping a new idea or scaling to production, llama-agents offers the tools you need to bring your AI vision to life. Check out the repo to learn more, especially our library of examples.We're excited to see what the community builds with llama-agents. Happy coding!Related articlesBuilding a multi-agent concierge system2024-07-17Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems2024-06-26Automate online tasks with MultiOn and LlamaIndex2024-05-23How to build LLM Agents in TypeScript with LlamaIndex.TS2024-02-08LlamaIndexBlogPartnersCareersContactStatusEnterpriseLlamaCloudLlamaParseOpen SourcePython packagePython docsTypeScript packageTypeScript docsLlamaHubGitHubCommunityNewsletterDiscordTwitter/XLinkedInYouTubeStarter projectscreate-llamaSEC InsightsChat LlamaIndexLlamaBotRAG CLI 2024 LlamaIndexPrivacy NoticeTerms of Service<br>

**Node ID:** 3058ab62-4cec-4b57-a570-cbc63c07b0be<br>**Similarity:** 0.63911602171461<br>**Text:** For instance, you may want to define a specific workflow over text-to-SQL with our NLStructStoreQueryEngine (constrained), or a router module to decide between semantic search or summarization (less constrained), or use our SubQuestionQueryEngine to decompose a question among sub-documents (even less constrained).By default, agent loops are unconstrained, and can theoretically reason over any set of tools that you give them. This means that you can get out-of-the-box advanced search/retrieval capabilities  for instance, in our OpenAI cookbook we show that you can get joint text-to-SQL capabilities by simply providing a SQL query engine and Vector Store Query engine as tools. But on the other hand, agents built in this fashion can be quite unreliable (see our blog post for more insights). If you are using agents for search/retrieval, be mindful of the 1) LLM you pick, and the 2) set of tools you pick too.How are LlamaIndex data agents different than existing agent frameworks (LangChain, Hugging Face, etc.)?Most of these core concepts are not new. Our overall design has taken inspiration from popular tools and frameworks for building agents. But in our data agents design, weve tried our best to answer the following key questions well:How do we effectively index/query and retrieve data beforehand?How do we effectively index/query and retrieve data on the fly?How do we design API interfaces for read/writes that are simultaneously rich (can take in structured inputs), but also easy for agents to understand?How do we properly get sources in citations?Our goal with data agents is to create automated knowledge workers that can reason over and interact with data. Our core toolkit provides the foundations for properly indexing, retrieving, and querying data  these can be easily integrated as tools. We provide some additional tool abstractions to handle the cases where you want to cache API outputs on the fly (see above). Finally, we provide principled tool abstractions and design principles so that agents can interface with external services in a structured manner.Can I use Tools with LangChain agents? You can easily use any of our tools with LangChain agents as well.tools = tool_spec.to_tool_list() langchain_tools = [t.to_langchain_tool() for t in tools]See our tools usage guide for more details!ConclusionIn summary, today we launched two key items: Data Agent components (incl. agent reasoning loop and tool abstractions) and the LlamaHub Tool repository.ResourcesWeve written a comprehensive section in the docs  take a look here: https://gpt-index.readthedocs.io/en/latest/core_modules/agent_modules/agents/root.htmlTake a look at our LlamaHub Tools section: https://llamahub.ai/Notebook Tutorials for LlamaHub Tools: https://github.com/emptycrown/llama-hub/tree/main/llama_hub/tools/notebooksIf you have questions, please hop on our Discord: https://discord.gg/dGcwcsnxhURelated articlesBuilding a multi-agent concierge system2024-07-17Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems2024-06-26Automate online tasks with MultiOn and LlamaIndex2024-05-23Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations2024-03-19LlamaIndexBlogPartnersCareersContactStatusEnterpriseLlamaCloudLlamaParseOpen SourcePython packagePython docsTypeScript packageTypeScript docsLlamaHubGitHubCommunityNewsletterDiscordTwitter/XLinkedInYouTubeStarter projectscreate-llamaSEC InsightsChat LlamaIndexLlamaBotRAG CLI 2024 LlamaIndexPrivacy NoticeTerms of Service<br>

**Node ID:** 3b9739f3-af80-476a-a621-a69d4e1a5540<br>**Similarity:** 0.6332048798838363<br>**Text:** Data Agents  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexJerry Liu  Jul 12, 2023Data AgentsLlamaindexNLPAIAgentsDataToday were incredibly excited to announce the launch of a big new capability within LlamaIndex: Data Agents.Data Agents are LLM-powered knowledge workers that can intelligently perform various tasks over your data, in both a read and write function. They are capable of the following:Perform automated search and retrieval over different types of data  unstructured, semi-structured, and structured.Calling any external service API in a structured fashion. They can either process the response immediately, or index/cache this data for future use.Storing conversation history.Using all of the above to fulfill both simple and complex data tasks.Weve worked hard to provide abstractions, services, and guides on both the agents side and tools side in order to build data agents. Todays launch consists of the following key components:General Agent/Tool Abstractions: a set of abstractions to build agent loops, and to have those loops interact with tools according to a structured API definition.LlamaHub Tool Repository: A brand-new section within LlamaHub that consists of 15+ Tools (e.g. Google Calendar, Notion, SQL, OpenAPI) that can be connected. Opening to community contributions!See below for full details. We show you how to build a Gmail agent thats able to automatically create/send emails in <10 lines of code!ContextOur core mission at LlamaIndex is to unlock the full capabilities of LLMs over your external sources of data. It provides a set of tools to both define state (how to parse/structure your data), and compute (how to query your data). Up until now, our framework has primarily focused on search and retrieval use case. We have an incredible suite of tools and capabilities that not only allow you to create the basic RAG stack around a vector database + top-k retrieval, but also offer much greater functionality beyond that.A lot of that technology used to lie in our query engines. Our goal was to increase the capability of query engines to answer a wide range of different queries. In order to do this, we had to improve the reasoning capabilities of these query engines. As a result some of our existing query capabilities contain agent-like components: we have query engines capable of chain-of-thought reasoning, query decomposition, and routing. In the process, users had the option of choosing from a spectrum of query engines that had more constrained reasoning capabilities to less constrained capabilities.But there was a huge opportunity for LLMs to have an even richer set of interactions with data; they should be capable of general reasoning over any set of tools, whether from a database or an API. They should also be capable of both read and write capabilities  the ability to not only understand state but also modify it. As a result they should be able to do more than search and retrieval from a static knowledge source.Some existing services, toolkits, and research papers have already demonstrated the possibilities of LLM-powered agents that can interact with the external environment. Using these existing approaches as inspiration, we saw an opportunity to build a principled series of abstractions enabling anyone to build knowledge workers over their data.Core Components of Data AgentsBuilding a data agent requires the following core components:A reasoning loopTool abstractionsAt a high-level, a data agent is provided with a set of APIs, or Tools, to interact with. These APIs can return information about the world, or perform an action that modifies state. Each Tool exposes a request/response interface. The request is a set of structured parameters, and the response can be any format (at least conceptually, in most cases the response here is a text string of some form).Given an input task, the data agent uses a reasoning loop to decide which tools to use, in which sequence, and the parameters to call each tool.<br>

**Node ID:** 87691d04-b40e-44c3-abfc-df4e157d5bcb<br>**Similarity:** 0.6313470492685472<br>**Text:** Lets bring in our dependencies and set up our control plane again:import dotenv dotenv.load_dotenv() from llama_agents import ( AgentService, AgentOrchestrator, ControlPlaneServer, SimpleMessageQueue, ) from llama_index.core.agent import FunctionCallingAgentWorker from llama_index.core.tools import FunctionTool from llama_index.llms.openai import OpenAI import logging # change logging level to enable or disable more verbose logging logging.getLogger("llama_agents").setLevel(logging.INFO) # create our multi-agent framework components message_queue = SimpleMessageQueue() control_plane = ControlPlaneServer( message_queue=message_queue, orchestrator=AgentOrchestrator(llm=OpenAI()), )Then as before we create a tool and an agent, though this time well add a second agent:# create a tool def get_the_secret_fact() -> str: """Returns the secret fact.""" return "The secret fact is: A baby llama is called a 'Cria'." tool = FunctionTool.from_defaults(fn=get_the_secret_fact) # create our agents worker1 = FunctionCallingAgentWorker.from_tools([tool], llm=OpenAI()) worker2 = FunctionCallingAgentWorker.from_tools([], llm=OpenAI()) agent1 = worker1.as_agent() agent2 = worker2.as_agent()We turn those agents into services:agent_server_1 = AgentService( agent=agent1, message_queue=message_queue, description="Useful for getting the secret fact.", service_name="secret_fact_agent", host="localhost", port=8003 ) agent_server_2 = AgentService( agent=agent2, message_queue=message_queue, description="Useful for getting random dumb facts.", service_name="dumb_fact_agent", host="localhost", port=8004 )And finally we launch each service as an independent agent. Here were doing them all from a single script, but each of these could be a totally separate service, launched and scaled independently:from llama_agents import ServerLauncher, CallableMessageConsumer # Additional human consumer def handle_result(message) -> None: print(f"Got result:", message.data) # the final result is published to a "human" consumer # so we define one to handle it! human_consumer = CallableMessageConsumer( handler=handle_result, message_type="human" ) # Define Launcher launcher = ServerLauncher( [agent_server_1, agent_server_2], control_plane, message_queue, additional_consumers=[human_consumer] ) launcher.launch_servers()Real-time monitoringOne of the coolest debugging features of our multi-agent system is our agent monitor, which is built right in. You launch it like this:llama-agents monitor --control-plane-url http://127.0.0.1:8000Once launched, you get an intuitive, point-and-click terminal application. You can see both of the agents running, and at the bottom you can inject a task like the query What is the secret fact? Youll get a job ID which you can then click on to see your results:Building a Query Rewriting RAG SystemLet's look at a more complex example: a Query Rewriting RAG system. This system will rewrite user queries to improve retrieval, then use the rewritten query to perform RAG over a document.This example demonstrates how to create a more sophisticated system that combines query rewriting with RAG to improve question-answering capabilities. See this notebook for a fuller explanation of whats going on.import dotenv dotenv.load_dotenv() # our .env defines OPENAI_API_KEY from llama_index.core import VectorStoreIndex, Document from llama_index.core.agent import FnAgentWorker from llama_index.core import PromptTemplate from llama_index.core.query_pipeline import QueryPipeline from llama_index.core.query_engine import RetrieverQueryEngine from llama_agents import ( AgentService, ControlPlaneServer, SimpleMessageQueue, PipelineOrchestrator, ServiceComponent, ) from llama_agents.launchers import LocalLauncher from llama_index.llms.openai import OpenAI import logging # change logging level to enable or disable more verbose logging logging.getLogger("llama_agents").setLevel(logging.INFO) # Load and index your document docs = [Document(text="The rabbit is a small mammal with long ears and a fluffy tail. His name is Peter.")]<br>

**Node ID:** 76606973-5bf0-47bd-9440-cd4989710607<br>**Similarity:** 0.6283408263769629<br>**Text:** LlamaIndex Newsletter 2024-07-02  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterpriseOpen sourceCommunityCareersBlog Talk to us 2024 LlamaIndexLlamaIndex  Jul 2, 2024LlamaIndex Newsletter 2024-07-02Hello, Llama enthusiasts! Welcome to this weeks edition of the LlamaIndex newsletter! In this issue, were excited to bring you exciting updates about llama-agents, live demos, extensive guides, and in-depth tutorials to enhance your understanding of our tools.Before moving into our newsletter, we have an exciting update on our enterprise offerings. We are thrilled to announce the waitlist release of LlamaCloud, our fully-managed ingestion service. Sign up now if youre eager to collaborate and build LLM applications with LlamaCloud. The highlights:Launched Llama-Agents Framework: Our new alpha-release, llama-agents, enables multi-agent AI systems for production with a distributed architecture, HTTP API communication, and agentic orchestration. Its designed for easy deployment, scalability, and observability. Blogpost, Tweet.create-llama Integrated with LlamaCloud: Streamline your LLM application data pipelines with create-llama, now integrated with LlamaCloud for faster setup and efficient system maintenance. Tweet. Feature Releases and Enhancements:We have launched llama-agents - new alpha-release framework that enables multi-agent AI systems to go into production. It features a distributed, service-oriented architecture, communication through standard HTTP APIs, agentic orchestration of flows, and is designed for easy deployment, scalability, and observability. Blogpost, Tweet.create-llama is now integrated with LlamaCloud to streamline the setup and management of data pipelines for LLM applications, providing a fast and efficient way to deploy and maintain these systems. Tweet.We have integrated with DSPy for Optimized RAG by utilizing DSPys optimization capabilities with LlamaIndexs data tools to enhance your query pipelines, optimize prompts, or repurpose DSPy predictors. Cookbook, Tweet. Demos:Automating Code Reviews, project by Composio with LlamaIndex automates code reviews using an AI agent in under 100 lines of code that monitors GitHub PRs, reviews them immediately upon creation, and posts feedback directly to your Slack channel. Codebase. Guides:Guide to Building an Agentic RAG Service with our comprehensive notebook that walks you through creating vector indexes, transforming them into query engines, turning each engine into a tool, providing these tools to agents, and launching the agents as services.Guide to AI Agents with LlamaIndex: Andreis comprehensive workshop from Gen AI Philippines, showcasing LLM applications through LlamaIndex. This beginner-friendly session covers topics from RAG to multi-hop agents. Video, Notebook. Tutorials:Kingzzms tutorial on crafting a custom hybrid retriever using LlamaIndexs flexible abstractions. This tutorial teaches you how to integrate full text and dense search capabilities from Elastic, and how to write your own reciprocal rank fusion function for optimal retrieval strategy.Jeffs tutorial on which outlines the essential tools needed to construct a report generator using a ReAct agent. Learn how to integrate a RAG tool over guideline documents, a web search tool, and a report generation tool that converts markdown text into PDFs.1littlecoders tutorial on llama-agents provides a detailed introduction to transforming multi-agent systems into microservices for production, including setup examples and a walkthrough of the architecture involving the control plane, message queue, and agent services using LlamaIndex abstractions.Mervin Praisons tutorial on the llama-agents framework provides a concise guide to setting up agent services, from notebook synchronization to server-client interactions, complete with over 10 practical examples.LlamaIndexBlogPartnersCareersContactStatusEnterpriseLlamaCloudLlamaParseOpen SourcePython packagePython docsTypeScript packageTypeScript docsLlamaHubGitHubCommunityNewsletterDiscordTwitter/XLinkedInYouTubeStarter projectscreate-llamaSEC InsightsChat LlamaIndexLlamaBotRAG CLI 2024 LlamaIndexPrivacy NoticeTerms of Service<br>

In [83]:
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(retriever, response_mode="refine")
base_query_engine = RetrieverQueryEngine.from_args(base_retriever, response_mode="refine")

response = query_with_sources(query_engine, query_str)


> Merging 2 nodes into parent node.
> Parent node id: 4d7dabcf-ee2e-41af-90a1-3f428d597844.
> Parent node text: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  Ll...

> Merging 1 nodes into parent node.
> Parent node id: e81e093e-bf96-419a-9acd-aea6a0265a5d.
> Parent node text: His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agen...

> Merging 1 nodes into parent node.
> Parent node id: 80ab5c6d-b8f9-46fa-aac6-7f1deec68513.
> Parent node text: LlamaIndex Newsletter 2024-07-02  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen s...

> Merging 2 nodes into parent node.
> Parent node id: df506443-a581-4826-9d00-63229ca12281.
> Parent node text: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  Ll...

> Merging 1 nodes into parent node.
> Parent node id: 71a33e83-9fdc-4ac6-a397-c046a2e638d4.
> Parent node text: LlamaIndex Newsletter 2024-07-02

**`Final Response:`** The key features of llama-agents include a Distributed Service Oriented Architecture where each agent can function as an independently running microservice, standardized API interfaces for communication between agents, the ability to define agentic and explicit orchestration flows, ease of deployment with the capability to launch, scale, and monitor each agent independently, and built-in observability tools for scalability and resource management.

In [85]:
base_response = query_with_sources(base_query_engine, query_str)

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems  LlamaIndex, Data Framework for LLM ApplicationsEnterpriseOpen sourceCommunityCareersBlog Talk to usEnterp 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: His name is Peter.")] index = VectorStoreIndex.from_documents(docs) # Define a query rewrite agent HYDE_PROMPT_STR = ( "Please rewrite the following query to include more detail:\n{query_str}\n" ) HYD 


Title: Data Agents
 URL: https://www.llamaindex.ai/blog/data-agents-eed797d7972f
Text preview

**`Final Response:`** The key features of llama-agents include a Distributed Service Oriented Architecture where each agent can function as an independently running microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows for interactions between agents, ease of deployment with the capability to launch, scale, and monitor each agent and control plane independently, and scalability and resource management through built-in observability tools to monitor system performance and quality.

## Re-ranker

In [86]:
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", None)


def get_retrieved_nodes(query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False):
    """
    Retrieve and optionally rerank nodes based on a query string.

    Args:
        query_str (str): The query string to search for.
        vector_top_k (int, optional): Number of top results to retrieve from the vector index. Defaults to 10.
        reranker_top_n (int, optional): Number of top results to keep after reranking. Defaults to 3.
        with_reranker (bool, optional): Whether to apply reranking. Defaults to False.

    Returns:
        List[NodeWithScore]: A list of retrieved (and optionally reranked) nodes with their scores.
    """


    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=5,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    """
    Display a pandas DataFrame as a formatted HTML table.

    Args:
        df (pandas.DataFrame): The DataFrame to display.

    Returns:
        IPython.core.display.HTML: An HTML representation of the DataFrame.
    """
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    """
    Visualize a list of retrieved nodes as a formatted table.

    This function creates a DataFrame from the nodes' scores and text content,
    then displays it using the pretty_print function.

    Args:
        nodes (List[NodeWithScore]): A list of retrieved nodes with their scores.

    Returns:
        None: This function displays the result but does not return a value.
    """
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

### Without re-rank

In [120]:
new_nodes = get_retrieved_nodes(
    "What are key features of llama-agents?",
    vector_top_k=5,
    with_reranker=False,
)

In [121]:
visualize_retrieved_nodes(new_nodes)

In [125]:
def generate_response(query, retrieved_nodes):
    # Initialize context and sources string
    context = ""
    sources = "Sources:\n"

    # Combine the text from all retrieved nodes and create sources information
    for i, node in enumerate(retrieved_nodes, 1):
        # Get the full text of the node
        full_text = node.node.get_text()

        title = node.node.metadata.get('title', 'Unknown Title')

        # Get the URL from metadata
        url = node.node.metadata.get('URL', 'URL not available')

        # Add to context
        context += f"{full_text}\n\n"

        # Add to sources
        sources += f"{i}. Title: {title}\n"
        sources += f" URL: {url}\n"
        sources += f"   Preview: {full_text[:200]}...\n\n"

    # Create a prompt that includes the context and the query
    prompt = f"Based on the following information:\n\n{context}\nPlease answer the question: {query}"

    # Generate a response using the query engine
    response = query_engine.query(prompt)

    # Combine the response with the sources information
    full_response = f"\n\n{sources}"
    display_response(response)
    print(full_response)


# Usage
query = "What are key features of llama-agents?"
new_nodes = get_retrieved_nodes(query, vector_top_k=5, with_reranker=False)
response = generate_response(query, new_nodes)

**`Final Response:`** The key features of llama-agents include a distributed service-oriented architecture where each agent can run independently as a microservice, communication through standardized API interfaces using a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment with the capability to launch, scale, and monitor each agent and control plane independently, and built-in observability tools for monitoring the quality and performance of the system and individual agent services.



Sources:
1. Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
   Preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd...

2. Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
   Preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp...

3. Title: Data Agents
 URL: https://www.llamaindex.ai/blog/data-agents-eed797d7972f
   

### With re-rank

In [116]:

new_nodes = get_retrieved_nodes(
    "What are key features of llama-agents?",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

In [117]:
visualize_retrieved_nodes(new_nodes)

In [126]:
def generate_response(query, retrieved_nodes):
    # Initialize context and sources string
    context = ""
    sources = "Sources:\n"

    # Combine the text from all retrieved nodes and create sources information
    for i, node in enumerate(retrieved_nodes, 1):
        # Get the full text of the node
        full_text = node.node.get_text()

        title = node.node.metadata.get('title', 'Unknown Title')

        # Get the URL from metadata
        url = node.node.metadata.get('URL', 'URL not available')

        # Add to context
        context += f"{full_text}\n\n"

        # Add to sources
        sources += f"{i}. Title: {title}\n"
        sources += f" URL: {url}\n"
        sources += f"   Preview: {full_text[:200]}...\n\n"

    # Create a prompt that includes the context and the query
    prompt = f"Based on the following information:\n\n{context}\nPlease answer the question: {query}"

    # Generate a response using the query engine
    response = query_engine.query(prompt)

    # Combine the response with the sources information
    full_response = f"\n\n{sources}"
    display_response(response)
    print(full_response)


# Usage
query = "What are key features of llama-agents?"
new_nodes = get_retrieved_nodes(query, vector_top_k=3, with_reranker=False)
response = generate_response(query, new_nodes)


**`Final Response:`** The key features of llama-agents include a distributed service-oriented architecture where each agent can run independently as a microservice, communication via standardized API interfaces using a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment allowing for independent launching, scaling, and monitoring of each agent and the control plane, as well as built-in observability tools for monitoring the quality and performance of the system and individual agent services.



Sources:
1. Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
   Preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd...

2. Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
   Preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp...

3. Title: Data Agents
 URL: https://www.llamaindex.ai/blog/data-agents-eed797d7972f
   

## HYDE

HyDE is a technique where given a natural language query, a hypothetical document/answer is generated first. This hypothetical document is then used for embedding lookup rather than the raw query.

In [ ]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

In [ ]:
query_str = "What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system"

### Without hyde

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display_response(response)

**`Final Response:`** The two main metrics used to evaluate the performance of the different rerankers in the RAG system are Hit Rate and Mean Reciprocal Rank (MRR).

### With HYDE

In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)

In [ ]:
display_response(response)

**`Final Response:`** The Open AI and Jina AI-Base embeddings, especially when paired with the Cohere Rerank/bge-reranker-large reranker, set the gold standard for both hit rate and MRR. The influence of rerankers, particularly Cohere Rerank/bge-reranker-large, cannot be overstated. They play a key role in improving the MRR for many embeddings, showing their importance in making search results better. Choosing the right embedding for the initial search is essential; even the best reranker can't help much if the basic search results aren't good. To get the best out of retrievers, it's important to find the right mix of embeddings and rerankers.

In [ ]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'The two main metrics used to evaluate the performance of the different rerankers in the RAG system are BLEU score and ROUGE score. BLEU score measures the similarity between the generated text and the reference text by comparing n-grams, while ROUGE score evaluates the quality of the generated text by measuring the overlap of n-grams and word sequences between the generated text and the reference text. These metrics help to assess the effectiveness of the rerankers in improving the relevance and coherence of the generated text in the RAG system. Additionally, other metrics such as perplexity, fluency, and diversity may also be used to provide a comprehensive evaluation of the rerankers\' performance."""'

The query_bundle contains the original query and custom_embedding_strs which is the hypothetical document generated by the LLM in response to the query

## Multi-step query


In [ ]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo",api_key=open_ai_key )

# LLM (gpt-4o-mini)
gpt4 = OpenAI(temperature=0, model="gpt-4o-mini",api_key=open_ai_key)

In [ ]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)

In [ ]:
index_summary = "Used to answer questions about llamaindex" #A string describing the purpose of the index

from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
    index_summary=index_summary,
)
response_gpt = query_engine.query(
    "What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?",
)

> Current query: What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?
> New query: What specific metrics are used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?
> Current query: What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?
> New query: What are the specific metrics used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?
> Current query: What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?
> New query: What is the significance of Answer Relevancy in evaluating RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?


In [ ]:
print(str(response_gpt))

Answer Relevancy and Context Relevancy are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook.


In [ ]:
sub_qa = response_gpt.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

[("What specific metrics are used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?", "Answer: Answer Relevancy, Context Relevancy, Faithfulness, Retrieval Evaluation, and Batch Evaluations with Batch Eval Runner are specific metrics used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook."), ("What are the specific metrics used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?", "Answer: Answer Relevancy, Context Relevancy, Faithfulness, Retrieval Evaluation, and Batch Evaluations with Batch Eval Runner are the specific metrics used to evaluate RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook."), ("What is the significance of Answer Relevancy in evaluating RAG system performance in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?", 'Answer Relevancy is cr

### Index Setup

## Router Query Engine

In [ ]:
vector_index = load_index_from_storage(storage_context, index_id="vector_index")

keyword_table_index = load_index_from_storage(storage_context, index_id="keyword_table_index")


In [ ]:
#selects one out of several candidate query engines to execute a query.
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

keyword_tool = QueryEngineTool(
    query_engine=keyword_table_index.as_query_engine(),
    metadata=ToolMetadata(
        name="keyword_search",
        description="Useful for finding documents containing specific keywords or phrases."
    )
)


### Single Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, keyword_tool],
    select_multi=False
)

query_with_sources(query_engine, "What are key features of llama-agents?")


Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp 




**`Final Response:`** The key features of llama-agents include a distributed service-oriented architecture where each agent can function as an independent microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment allowing for launching, scaling, and monitoring each agent independently, and built-in observability tools for monitoring the quality and performance of the system and individual agent services.

### Multi Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, keyword_tool],
    select_multi=True #allows the query engine to gather information from different sources (in this case, both the vector_tool and keyword_tool) and combine their outputs to form a more comprehensive answer.
)

query_with_sources(query_engine, "What are key features of llama-agents?")

Sources:
Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: his name is peter.")] index = vectorstoreindex.from_documents(docs) # define a query rewrite agent hyde_prompt_str = ( "please rewrite the following query to include more detail:\n{query_str}\n" ) hyd 


Title: Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems
 URL: https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems
Text preview: introducing llama-agents: a powerful framework for building production multi-agent ai systems  llamaindex, data framework for llm applicationsenterpriseopen sourcecommunitycareersblog talk to usenterp 




**`Final Response:`** The key features of llama-agents include a distributed service-oriented architecture where each agent can function as an independent microservice, communication via standardized API interfaces facilitated by a central control plane orchestrator, the ability to define agentic and explicit orchestration flows, ease of deployment allowing for launching, scaling, and monitoring each agent and the control plane independently, as well as built-in observability tools for monitoring the quality and performance of the system and individual agent services.

## Backoff

In [ ]:
import logging.config
import random
import time
from openai import OpenAI, RateLimitError

logging.basicConfig(level=logging.INFO)

def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 15,
    errors: tuple = (RateLimitError,),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specific errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                logging.info(f"Rate limit hit. Retrying in {delay:.2f} seconds...")
                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper

@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return openAI_client.chat.completions.create(**kwargs)

openAI_client = OpenAI()

response = completions_with_backoff(
            model="gpt-4o",
            messages=[{"role": "user", "content": "I am Phanh, who are you?"}],
            n=1,
            temperature=0
        )

print(response.choices[0].message.content)


Hello Phanh! I'm an AI language model created by OpenAI. How can I assist you today?


#### Customize query engine using Backoff

In [ ]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

qa_prompt = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)


class RAGStringQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    @retry_with_exponential_backoff
    def custom_query(self, query_str: str):
        print("===custom query in progress===")
        nodes = self.retriever.retrieve(query_str)

        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)

In [ ]:
synthesizer = get_response_synthesizer(response_mode="refine")

query_engine = RAGStringQueryEngine(
    retriever=vector_index.as_retriever(),
    response_synthesizer=synthesizer,
    llm = OpenAI(model="gpt-3.5-turbo"),
    qa_prompt=qa_prompt,
)

In [ ]:
response = query_engine.query("What are key features of llama-agents?")
display_response(response)

===custom query in progress===


**`Final Response:`** Some key features of llama-agents include:
1. Distributed service oriented architecture: Each agent can be its own independently running microservice, orchestrated by a customizable llm-powered control plane.
2. Communication via standardized API interfaces: Agents can interface with each other using a central control plane orchestrator and pass messages through a message queue.
3. Define agentic and explicit orchestration flows: Developers have the flexibility to define the sequence of interactions between agents or leave it up to an agentic orchestrator.
4. Ease of deployment: Agents and the control plane can be launched, scaled, and monitored independently.
5. Scalability and resource management: Built-in observability tools allow monitoring of the system's quality and performance, as well as each individual agent service.